In [18]:
# импорт библиотек

import torch
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import torchvision
import torchvision.transforms as transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#### Полносвязная модель для датасета MNIST

In [9]:

#  Входной размер: in_dim;
#  Количество нейронов в первом слое нейронной сети n_hidden_1;
#  Количество нейронов во втором слое нейронной сети n_hidden_2, out_dim
#  Количество нейронов в третьем слое сети (вывод в)

class simpleNet(nn.Module):
    def __init__(self,in_dim,n_hidden_1,n_hidden_2,out_dim):
        super(simpleNet,self).__init__()
        self.layer1 = nn.Linear(in_dim,n_hidden_1)
        self.layer2 = nn.Linear(n_hidden_1,n_hidden_2)
        self.layer3= nn.Linear(n_hidden_2,out_dim)
        
    def forward(self,x):
        hidden_1_out = self.layer1(x)
        hidden_2_out = self.layer2(hidden_1_out)
        out = self.layer3(hidden_2_out)
        return out



In [13]:
# гиперпараметры
batch_size = 100
learning_rate = 0.01


data_tf = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize([0.5],[0.5])])
train_dataset = datasets.MNIST(root='./data',train=True,transform=data_tf,download=True)
test_dataset = datasets.MNIST(root='./data',train=False,transform=data_tf)

train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=batch_size,shuffle=False)


# Размер входного изображения - 28 * 28; два скрытых слоя - 300 и 100 соответственно; конечный размер вывода - 10
model = simpleNet(28*28,300,100,10)  

if torch.cuda.is_available():
    model = model.cuda()

criterion = nn.CrossEntropyLoss()  # Перекрестная потеря энтропии
optimizer = optim.Adagrad(model.parameters(),lr=learning_rate) # Функция оптимизации

# Тренировочная модель
epoch = 0
for data in train_loader:
    img, label = data
    img = img.view(img.size(0), -1)
    if torch.cuda.is_available():
        img = img.cuda()
        label = label.cuda()
    else:
        img = Variable(img)
        label = Variable(label)
    out = model(img)
    loss = criterion(out, label)
    print_loss = loss.data.item()
 
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    epoch+=1
    if epoch%50 == 0:
        print(f'epoch: {epoch},Train Loss:{loss.data.item():.6f}')  

#Model Evaluation
model.eval()
eval_loss = 0
eval_acc = 0
for data in test_loader:
    img,label = data
    img = img.view(img.size(0),-1)  
    if torch.cuda.is_available():
        img = Variable(img,volatile=True).cuda()
        label = Variable(label,volatile=True).cuda()
    else:
        img = Variable(img,volatile=True)
        label = Variable(label,volatile=True)
    out = model(img)
    loss = criterion(out,label)
    eval_loss += loss.data.item() * label.size(0)
    _,pred = torch.max(out,1)
    num_correct = (pred==label).sum()
    eval_acc += num_correct.data.item()
print(f'Test Loss:{eval_loss/(len(test_dataset)):.6f},Acc:{eval_acc/(len(test_dataset)):.6f}')
    



epoch: 50,Train Loss:0.463890
epoch: 100,Train Loss:0.388615
epoch: 150,Train Loss:0.493248
epoch: 200,Train Loss:0.481885
epoch: 250,Train Loss:0.285333
epoch: 300,Train Loss:0.508131
epoch: 350,Train Loss:0.368897
epoch: 400,Train Loss:0.285408
epoch: 450,Train Loss:0.222345
epoch: 500,Train Loss:0.255201
epoch: 550,Train Loss:0.459309
epoch: 600,Train Loss:0.600211


/var/folders/rw/fy4jnmhn3l72v47ky8bt9r9w0000gn/T/ipykernel_67098/1025919584.py:56: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  img = Variable(img,volatile=True)
/var/folders/rw/fy4jnmhn3l72v47ky8bt9r9w0000gn/T/ipykernel_67098/1025919584.py:57: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  label = Variable(label,volatile=True)


Test Loss:0.316099,Acc:0.908900


#### Сверточная модель для датасета MNIST

In [20]:
# гиперпараметры
num_epochs = 4
batch_size = 4
learning_rate = 0.001


# MNIST
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')
n_classes = len(classes)


# nn
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.fc1 = nn.Linear(16*4*4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*4*4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = ConvNet().to(device)


# функция потерь и оптимизатор
criterion = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate)


# train
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    
    for i, (images, labels) in enumerate(train_loader):
        
        images = images.to(device)
        labels = labels.to(device)
        
        # forward
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # backwards
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
        
        if (i+1) % 2000 == 0:
            print(f'epoch {epoch+1} / {num_epochs}, step {i+1} / {n_total_steps}, loss = {loss.item():.4f}')


# test
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(n_classes)]
    n_class_samples = [0 for i in range(n_classes)]
    
    for images, labels in test_loader:
        labels = labels.to(device)
        outputs = model(images)
        
        # max returns (value, index)
        _, predictions = torch.max(outputs, 1)
        n_samples += labels.shape[0]
        n_correct += (predictions == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predictions[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1
        
    acc = 100.0 * n_correct / n_samples
    print(f'accuracy = {acc} %\n')
    
    for i in range(n_classes):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'accuracy of {classes[i]}: {acc:.2f} %')

epoch 1 / 4, step 2000 / 15000, loss = 0.1087
epoch 1 / 4, step 4000 / 15000, loss = 0.0003
epoch 1 / 4, step 6000 / 15000, loss = 0.0018
epoch 1 / 4, step 8000 / 15000, loss = 0.0124
epoch 1 / 4, step 10000 / 15000, loss = 0.0002
epoch 1 / 4, step 12000 / 15000, loss = 0.0038
epoch 1 / 4, step 14000 / 15000, loss = 0.0071
epoch 2 / 4, step 2000 / 15000, loss = 0.3147
epoch 2 / 4, step 4000 / 15000, loss = 0.0258
epoch 2 / 4, step 6000 / 15000, loss = 0.0009
epoch 2 / 4, step 8000 / 15000, loss = 0.0002
epoch 2 / 4, step 10000 / 15000, loss = 0.0545
epoch 2 / 4, step 12000 / 15000, loss = 0.0011
epoch 2 / 4, step 14000 / 15000, loss = 0.0169
epoch 3 / 4, step 2000 / 15000, loss = 0.1524
epoch 3 / 4, step 4000 / 15000, loss = 0.0026
epoch 3 / 4, step 6000 / 15000, loss = 0.0005
epoch 3 / 4, step 8000 / 15000, loss = 0.0122
epoch 3 / 4, step 10000 / 15000, loss = 0.1437
epoch 3 / 4, step 12000 / 15000, loss = 0.1464
epoch 3 / 4, step 14000 / 15000, loss = 0.0044
epoch 4 / 4, step 2000 / 